# Final Project - Task 1: Barebone MLP
In this task, you will build a Multi-Layer Perceptron (MLP) from first principles by implementing both the forward and backward passes yourselves.

## Instructions:
- Each numbered `# TODO-k` marks a line of code that you are required to complete. Make sure you replace the `...` with your code. 
    - It's recommended to start each `TODO` by reviewing the relevant material and deriving the equation with good old pen and paper. Only then move on to the implementation.
- After some of the `TODO`s, there are check cells. You only need to run these cells, and no new code is required. These cells will verify that your implementation is correct. If your code is correct, the check cell should run without errors.
- Do not modify any other parts of the code. Only fill in the sections marked with `# TODO`.
- You are only allowed to use the `NumPy` library in your answers. Do not use PyTorch, TensorFlow, scikit-learn, or any other libraries.
    - Some checks and the last part use PyTorch and require no code to be written

- Once all TODOs are completed:
    - run ALL cells (even pre-written ones)
    - The model should successfully train on a synthetic regression task.
    - The final Mean Squared Error (MSE) should be less than 1e-3.

- Submit your completed notebook along with answers to the theoretical questions provided in the project report.

## Additional Notes:
- Read the explanations before each section carefully.
- Pay close attention to what each function receives and returns. Shape annotations are provided to guide your implementation.
- For those who want, a Linear algebra (e.g. what is a gradient) refresher is available [here](https://stanford.edu/~shervine/teaching/cs-229/refresher-algebra-calculus). Consulting with LLMs is advised as well!

By completing this task, you will have implemented:
- Linear layers (forward and backward)
- ReLU activation functions (forward and backward)
- Mean Squared Error loss (forward and backward)
- A simple training loop using stochastic gradient descent (SGD)

# A. Multilayer Perceptron (MLP)

An **MLP** (also known as feedforward) is a chain of linear transformations and activation functions applied to an input vector. This class of functions serves as a potent hypothesis class. As such, multilayered perceptrons form the basis of deep learning and are applicable across a vast set of diverse domains.


## One Input, One Hidden Layer

Suppose the input is a scalar $x \in \mathbb{R}$, and we want to compute 3 hidden features ($h_1$, $h_2$, $h_3$). Each feature is computed using:

$$
h_i = a(\theta_{i0} + \theta_{i1} \cdot x)
$$

For example, all three:

$$
\begin{aligned}
h_1 &= a(\theta_{10} + \theta_{11} x) \\
h_2 &= a(\theta_{20} + \theta_{21} x) \\
h_3 &= a(\theta_{30} + \theta_{31} x)
\end{aligned}
$$

This can be written more compactly using vectors and matrices:

$$
\mathbf{h} = a\left(
\begin{bmatrix}
\theta_{10} \\
\theta_{20} \\
\theta_{30}
\end{bmatrix}
+
\begin{bmatrix}
\theta_{11} \\
\theta_{21} \\
\theta_{31}
\end{bmatrix}
x
\right)
= a(\boldsymbol{\theta}_0 + \boldsymbol{\theta} \cdot x)
$$


## Multiple featuers

If $\mathbf{x} \in \mathbb{R}^3$ (e.g., $x = [x_1, x_2, x_3]$), and we want 3 hidden units, each one is:

$$
h_i = a(\theta_{i0} + \theta_{i1} x_1 + \theta_{i2} x_2 + \theta_{i3} x_3)
$$

Written as a matrix:

$$
\mathbf{h} = a\left(
\begin{bmatrix}
\theta_{10} \\
\theta_{20} \\
\theta_{30}
\end{bmatrix}
+
\begin{bmatrix}
\theta_{11} & \theta_{12} & \theta_{13} \\
\theta_{21} & \theta_{22} & \theta_{23} \\
\theta_{31} & \theta_{32} & \theta_{33}
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix}
\right)
= a(\boldsymbol{\theta}_0 + \boldsymbol{\Theta} \mathbf{x})
$$


## Multiple Samples (Batch Input)
Now we have:
* A Single input vector $\mathbf{x} \in \mathbb{R}^3$
* A batch $X \in \mathbb{R}^{B \times 3}$, where each row is an input sample
Suppose we have a **batch** of $B$ input vectors, each of dimension 3. We stack them into a matrix:

$$
X =
\begin{bmatrix}
x_1^{(1)} & x_2^{(1)} & x_3^{(1)} \\
x_1^{(2)} & x_2^{(2)} & x_3^{(2)} \\
\vdots    & \vdots    & \vdots    \\
x_1^{(B)} & x_2^{(B)} & x_3^{(B)}
\end{bmatrix}
\in \mathbb{R}^{B \times 3}
$$

Now, we can calclaute the 3 hidden units in parallel using matrix multiplication. This means, we calculate the values in each layer for all samples at the same time, and this is why our output now has multiple rows. The parameters are:

* Weight matrix $\boldsymbol{\Theta} \in \mathbb{R}^{3 \times 3}$
* Bias vector $\boldsymbol{\theta}_0 \in \mathbb{R}^{3}$

In matrix form, for all $B$ samples:

$$
H =
a\left(
\begin{bmatrix}
x_1^{(1)} & x_2^{(1)} & x_3^{(1)} \\
x_1^{(2)} & x_2^{(2)} & x_3^{(2)}
\end{bmatrix}
\begin{bmatrix}
\theta_{11} & \theta_{21} & \theta_{31} \\
\theta_{12} & \theta_{22} & \theta_{32} \\
\theta_{13} & \theta_{23} & \theta_{33}
\end{bmatrix}
+
\begin{bmatrix}
\theta_{10} & \theta_{20} & \theta_{30}
\end{bmatrix}
\right)
\in \mathbb{R}^{2 \times 3}
$$

So:

* Each row of $H$ is a hidden representation for one sample
* Each column of $\boldsymbol{\Theta}$ defines weights for one input feature across all hidden units
* Bias is broadcast across the batch


## Final Formulation

If we denote:

* $X \in \mathbb{R}^{B \times \text{in\_dim}}$
* $W = \boldsymbol{\Theta} \in \mathbb{R}^{\text{out\_dim} \times \text{in\_dim}}$
* $b = \boldsymbol{\theta}_0 \in \mathbb{R}^{\text{out\_dim}}$

Then the full batched layer is:

$$
Z = X W^\top + b \quad \in \mathbb{R}^{B \times \text{out\_dim}}
$$

$$
H = a(Z)
$$

This is exactly the form used in modern deep learning frameworks: batched input matrix, weight matrix, bias vector, and optional activation.


## Next Layers Use Same Pattern

The output of the first layer, $\mathbf{h}$, becomes the input to the next:

$$
\mathbf{h}' = a(\boldsymbol{\psi}_0 + \boldsymbol{\Psi} \cdot \mathbf{h})
$$

And so on.

## Final Output Layer (No Activation)

The last layer usually skips the activation (especially in regression):

$$
\mathbf{y} = \boldsymbol{\phi}^\top \cdot \mathbf{h}' + \phi_0
$$

## Summary (General Form)

Each layer applies the same operation:

$$
\mathbf{h}^{(l)} = a\left(\mathbf{h}^{(l-1)} \mathbf{W}^{(l)\top} + \mathbf{b}^{(l)\top}\right)
$$

Where:

* $\mathbf{W}^{(l)}$ is a matrix of weights
* $\mathbf{b}^{(l)}$ is a vector of biases
* $a$ is the activation function (e.g., ReLU)

# B. Terminology: Forward and Backward Passes

**Forward Pass**  
The *forward pass* refers to the sequence of computations that occur from the input layer to the output layer of a neural network. During this phase, the model generates predictions (`y_hat`) based on current weights and biases.

At each layer:
- Inputs are transformed using the layer’s parameters.
- The output of one layer becomes the input to the next.

For example, a linear layer followed by ReLU:
- $z = xW^\top + b$
- $a = \text{ReLU}(z)$

The forward pass is also used to compute the loss, which quantifies how far the predictions are from the true labels.

**Backward Pass (Backpropagation)**  
The *backward pass* is the process of computing gradients of the loss function with respect to the model’s parameters. This is done using the chain rule of calculus, applied from the output layer back through the network.

At each layer:
- The gradient of the loss is propagated backward.
- Gradients with respect to weights, biases, and inputs are computed.

These gradients are then used to update the parameters using an optimization algorithm such as stochastic gradient descent (SGD).

For example:
- Given $\frac{\partial \text{Loss}}{\partial a}$, compute:
  - $\frac{\partial \text{Loss}}{\partial W}$
  - $\frac{\partial \text{Loss}}{\partial b}$
  - $\frac{\partial \text{Loss}}{\partial x}$

The backward pass is essential for learning, as it tells the model how to adjust its parameters to reduce the loss.

## 1  Linear layer
### 1.1 Linear Forward `TODO-1`
In the function `linear_forward` fill in the missing code to calculate `z`, the pre-activation output of the a single layer.

In [ ]:
import numpy as np

def linear_forward(x, params):
    """
    Forward pass through a linear (fully connected) layer.

    Parameters:
        x (ndarray): Input of shape (batch_size, input_dim)
        params (dict): Contains:
            - "W": Weights of shape (output_dim, input_dim)
            - "b": Biases of shape (output_dim,)

    Returns:
        z (ndarray): Output of shape (batch_size, output_dim)
        cache (tuple): (x, params), cached for backward pass
    """
    W, b = params["W"], params["b"]
    
    # TODO-1: implement forward linear pass
    # === Your code here ===
    z = ...
    # === Your code end ===

    cache = (x, params)
    return z, cache


def initialize_linear_layer(input_dim, output_dim):
    """
    Initialize weights and biases for a linear (fully connected) layer using He initialization.

    Returns a dictionary containing:
        - "W": Weight matrix (output_dim, input_dim)
        - "b": Bias vector (output_dim,)
        - "dW": Gradient of weights (same shape as W), initialized to zeros
        - "db": Gradient of biases (same shape as b), initialized to zeros
    """
    return {
        "W": np.random.randn(output_dim, input_dim) * (2 / input_dim) ** 0.5,
        "b": np.zeros(output_dim),
        "dW": np.zeros((output_dim, input_dim)),
        "db": np.zeros(output_dim),
    }


### Check Cell
Run the cell below to see if your linear_forward is correctly implemented

In [ ]:
# === Check for TODO-1: linear_forward ===
#
#  Tests your implementation using a 2x3 input and a 2x3 weight matrix.

# Input matrix: shape (batch_size=2, input_dim=3)
x_test = np.array([
    [1.0, 2.0, 3.0],
    [0.5, -1.0, 2.0]
])

# Parameters:
# W has shape (output_dim=2, input_dim=3)
# b has shape (output_dim=2,)
params_test = {
    "W": np.array([
        [0.1, 0.2, 0.3],
        [-0.5, 0.0, 0.5]
    ]),
    "b": np.array([0.1, -0.2])
}

# Resulting shape: (2, 2)
expected_z = np.array([
    [1.5, 0.8],
    [0.55, 0.55]
])

# Run student code
z_out, _ = linear_forward(x_test, params_test)

# Check
assert np.allclose(z_out, expected_z), f"Incorrect result.\nGot:\n{z_out}\nExpected:\n{expected_z}"
print("✅ linear_forward passed the 2x3 input test.")


### 1·2 Linear backward `TODO-2`
In the backward pass of a linear (fully connected) layer, you are given the gradient of the loss with respect to the layer's output $\partial L / \partial z$, and are required to calculate the gradient of the loss with respect to the model paramters and inputs: $\partial L / \partial x$, $\partial L / \partial W$, $\partial L / \partial b$ using the chain rule.

![bpmlp.png](bpmlp.png)

| Symbol      | Meaning                                             | Shape                                      |
|-------------|------------------------------------------------------|---------------------------------------------|
| `x`         | Input to the layer                                   | $(B,\ \text{in\_dim})$                       |
| `W`         | Weight matrix                                        | $(\text{out\_dim},\ \text{in\_dim})$         |
| `b`         | Bias vector                                          | $(\text{out\_dim})$                          |
| `z`         | Pre-activation output                                | $(B,\ \text{out\_dim})$                      |
| `a`         | Post-activation output                               | $(B,\ \text{out\_dim})$                      |
| `y_hat`     | Model predictions                                    | $(B,\ \text{out\_dim})$                      |
| `y`         | Ground truth targets                                 | $(B,\ \text{out\_dim})$                      |
| `dz`        | $\partial L / \partial z$: gradient of loss w.r.t. layer output | $(B,\ \text{out\_dim})$           |
| `dx`        | $\partial L / \partial x$: gradient of loss w.r.t. input        | $(B,\ \text{in\_dim})$            |
| `dW`        | $\partial L / \partial W$: gradient of loss w.r.t. weights      | $(\text{out\_dim},\ \text{in\_dim})$ |
| `db`        | $\partial L / \partial b$: gradient of loss w.r.t. biases       | $(\text{out\_dim})$               |


**Notes:**
- Remember that the loss is a mean 
    - $\ell^{(i)}$: The **loss for a single sample** $i$
    - $L$: The **total loss**, averaged over the batch: $L = \frac{1}{B} \sum_{i=1}^B \ell^{(i)} = \frac{1}{B} \sum_{i=1}^B l(\hat{y}_1 - y_1)$

> Think of $\ell^{(i)}$ as how "wrong" the network is on one example,
> and $L$ as how wrong it is **on average** across the batch.

- $B$ is the batch size (i.e., number of samples in the input $x$).
- `in_dim` is the number of input features.
- `out_dim` is the number of output features (e.g., neurons in the layer)
- All gradients are computed during the **backward pass** and used to update the parameters during training.


**Hints:**
- Use `.T` for transposing when needed.
- Divide `dW`,`db` by the batch size $B$ (to average the gradients)
- Do not divide `dx` by the batch size $B$, since $x$ is not a paramter.
- Match the expected output shapes exactly.

In [ ]:
def linear_backward(dz, cache):
    """
    Backward pass through a linear layer.

    Parameters:
        dz (ndarray): Gradient of loss w.r.t. output z, shape (batch_size, output_dim)
        cache (tuple): Contains:
            - x (ndarray): Input from forward pass, shape (batch_size, input_dim)
            - params (dict): Layer parameters

    Returns:
        dx (ndarray): Gradient of loss w.r.t. input x, shape (batch_size, input_dim)
    """
    x, params = cache
    W = params["W"]

    # TODO-2: compute gradients
    # === Your code here ===
    dW = ...
    db = ...
    dx = ...
    # === Your code end ===

    params["dW"] = dW
    params["db"] = db    
    return dx

Check-Cell

In [ ]:
import torch, numpy as np

B, in_dim, out_dim = 4, 5, 3

x_t = torch.randn(B, in_dim,  requires_grad=True)
W_t = torch.randn(out_dim, in_dim, requires_grad=True)
b_t = torch.randn(out_dim,      requires_grad=True)

# Forward + backward in PyTorch (loss = SUM, not mean)
z_t   = x_t @ W_t.T + b_t
loss  = z_t.sum()                # ∂L/∂z = 1
loss.backward()

# Build NumPy copies
x   = x_t.detach().numpy()
W   = W_t.detach().numpy()
b   = b_t.detach().numpy()
dz  = np.ones_like(z_t.detach().numpy())    # upstream gradient = 1  (matches loss = sum)

params = {"W": W.copy(),
          "b": b.copy(),
          "dW": np.zeros_like(W),
          "db": np.zeros_like(b)}
cache  = (x, params)


# Compare 
dx = linear_backward(dz, cache)   # uses /B inside
np.testing.assert_allclose(params["dW"], W_t.grad.numpy() / B, rtol=1e-5)
np.testing.assert_allclose(params["db"], b_t.grad.numpy() / B, rtol=1e-5)
np.testing.assert_allclose(dx,            x_t.grad.numpy(),     rtol=1e-5)

print("✅ linear_backward passes autograd test with your current implementation.")

## 2 ReLU Activation

After the linear transformation, we apply a nonlinearity.
In this exercise, we use the **ReLU (Rectified Linear Unit)** function:

$$
\text{ReLU}(z) = \max(0,\ z)
$$

This function is applied **element-wise**, meaning:

* Each individual number is replaced by itself if it’s positive,
* Or by 0 if it’s negative.

### Example

Suppose the output of the linear layer for a batch of 2 samples is:

$$
Z =
\begin{bmatrix}
-1.2 & 0.0 & 3.4 \\
2.1 & -0.5 & 1.3
\end{bmatrix}
$$

Then applying ReLU element-wise gives:

$$
A = \text{ReLU}(Z) =
\begin{bmatrix}
0.0 & 0.0 & 3.4 \\
2.1 & 0.0 & 1.3
\end{bmatrix}
$$


In the backward part, we must return `da` which is: $\partial L/\partial a$


In [ ]:
def relu_forward(z):
    """
    Forward pass through a ReLU activation (element-wise max with 0).

    Parameters
    ----------
    z : ndarray, shape (batch_size, features)
        Pre-activation values coming from the previous linear layer

    Returns
    -------
    out   : ndarray, shape (batch_size, features)
        Post-activation values
    cache : ndarray
        A copy of the input `a`, saved so the backward pass
        can determine which entries were “active”.
    """

    # TODO-3: implement forward ReLU pass
    # === Your code here ===
    out = ...  # element-wise ReLU
    # === Your code end ===

    return out, z            # cache the raw input


def relu_backward(dout, a):
    """
    Backward pass for a ReLU layer.

    Parameters
    ----------
    dout : ndarray, shape (batch_size, features)
        Gradient of the loss with respect to the ReLU output.
    a    : ndarray, shape (batch_size, features)
        Same tensor that was given as `a` to `relu_forward`
        (cached during the forward pass).

    Returns
    -------
    da : ndarray, shape (batch_size, features)
        Gradient of the loss with respect to the ReLU input.
    """
    dz = dout.copy()         # keep upstream gradient
    # TODO-4: apply ReLU mask
    # === Your code here ===
    da = ...             # Hint: mask dz where a <= 0
    # === Your code end ===
    return da

Check-cell

In [ ]:
# === Check for TODO-3 and TODO-4: ReLU forward / backward ===
import numpy as np

# Tiny 2×4 tensor with positive, zero, and negative entries
z_test = np.array([[ 1.0, -2.0,  0.0, 3.0],
                   [-1.5,  2.5, -0.3, 0.0]], dtype=float)

# ---------- Forward check ----------
out, cache = relu_forward(z_test)

expected_forward = np.array([[1.0, 0.0, 0.0, 3.0],
                             [0.0, 2.5, 0.0, 0.0]], dtype=float)

np.testing.assert_array_equal(out, expected_forward)
print("✅ ReLU forward passes simple mask test.")

# ---------- Backward check ----------
# Upstream gradient: ones, same shape
dout = np.ones_like(z_test)

grad = relu_backward(dout, cache)

# Should propagate 1 where z_test>0, else 0
expected_grad = np.array([[1.0, 0.0, 0.0, 1.0],
                          [0.0, 1.0, 0.0, 0.0]], dtype=float)

np.testing.assert_array_equal(grad, expected_grad)
print("✅ ReLU backward passes simple mask test.")


## 3 Loss: Mean-Squared Error (MSE)

Fill in the **forward** and **backward** code blocks for the MSE loss layer.

#### Forward pass

*Inputs*

* $y_{\text{hat}}\in\mathbb{R}^{B\times d}$ — model predictions
* $y\in\mathbb{R}^{B\times d}$ — ground-truth targets

*Compute*

1. The element-wise residual matrix

   $$
     z\;=\;y_{\text{hat}}-y
     \qquad\bigl(\text{same shape }B\times d\bigr)
   $$

2. The loss value

   $$
     L = \operatorname{MSE}(y_{\text{hat}},y)
   $$


#### Backward pass

Produce the gradient with respect to the predictions, i.e.

$$
  g \;=\;\frac{\partial L}{\partial y_{\text{hat}}}
  \quad\in\mathbb{R}^{B\times d}.
$$

Return this tensor `g` so it can be propagated to earlier layers.


**Notation**

| quantity | meaning                              | shape       |
| -------- | ------------------------------------ | ----------- |
| $z$      | residuals $y_{\text{hat}}-y$         | $B\times d$ |
| $L$      | MSE loss                             | scalar      |
| $g$      | $\partial L/\partial y_{\text{hat}}$ | $B\times d$ |


In [ ]:
def mse_forward(y_hat, y):
    """
    Mean-Squared-Error forward pass.

    Parameters
    ----------
    y_hat : ndarray, shape (batch_size, out_dim)
        Model predictions.
    y      : ndarray, shape (batch_size, out_dim)
        Ground-truth targets.

    Returns
    -------
    loss : float
        Scalar MSE value
    diff : ndarray, shape (batch_size, out_dim)
        Residuals cached for the backward step.
    """
    # TODO-5: compute mean squared error loss
    # === Your code here ===
    diff = ...                  # (B, d)
    loss = ...         # scalar
    # === Your code end ===
    return loss, diff


def mse_backward(diff):
    """
    Mean-Squared-Error backward pass.

    Parameters
    ----------
    diff : ndarray, shape (batch_size, out_dim)
        Residuals from mse_forward (i.e., y_hat − y).

    Returns
    -------
    grad : ndarray, shape (batch_size, out_dim)
        Gradient of the loss with respect to y_hat.
    """
    batch_size = diff.shape[0]

    # TODO-6: compute gradient of the loss w.r.t. y_hat
    # === Your code here ===
    g = ...
    # === Your code end ===
    return g


Check-cell

In [ ]:
# Tiny deterministic example
y_hat = np.array([[3.0, 2.0],
                  [1.0, 4.0]], dtype=float)   # (B=2, d=2)

y_true = np.array([[2.0, 0.0],
                   [1.0, 5.0]], dtype=float)

loss, diff = mse_forward(y_hat, y_true)

expected_diff  = np.array([[ 1.0,  2.0],
                           [ 0.0, -1.0]], dtype=float)

expected_loss  = ((expected_diff**2).mean())  

np.testing.assert_array_equal(diff, expected_diff)
np.testing.assert_allclose(loss, expected_loss, rtol=1e-12)
print("✅ mse_forward produces correct residuals and loss.")

grad = mse_backward(diff)

expected_grad = np.array([[ 1.0,  2.0],
                          [ 0.0, -1.0]], dtype=float)

np.testing.assert_array_equal(grad, expected_grad)
print("✅ mse_backward returns correct gradient.")

## 4  Gradient descent
You're implementing the parameter update step in training. After computing gradients during backpropagation, this function updates the model's weights and biases

For each layer:

* `W` and `b` hold the current weights and biases.
* `dW` and `db` hold their respective gradients.

This step nudges the parameters in the direction that reduces the loss.

The `zero_grads` function clears out the gradients **after** the step, so the next minibatch starts fresh. Without it, gradients from multiple steps would accumulate and give incorrect updates.

In [ ]:
def sgd_step(param_list, lr):
    """
    Perform SGD update step on all parameters.

    Parameters:
        param_list (list): List of parameter dicts with keys "W", "b", "dW", "db"
        lr (float): Learning rate
    """
    for p in param_list:
        W = p["W"]
        dw = p["dW"]
        b = p["b"]
        db = p["db"]

        # TODO-7: implement SGD step
        # === Your code here ===
        W = ...
        b = ...
        # === Your code end ===

        p["W"] = W            # update weight
        p["b"] = b            # update bias

def zero_grads(param_list):
    """
    Zero out all gradients in the parameter list.

    Parameters:
        param_list (list): List of parameter dicts
    """
    for params in param_list:
        params["dW"].fill(0)
        params["db"].fill(0)

## 5  Build the MLP

In [ ]:
def build_mlp(input_dim, hidden_dims, output_dim):
    """
    Build a list of layers representing the MLP.

    Parameters:
        input_dim (int): Dimension of input features
        hidden_dims (list): List of integers for hidden layer sizes
        output_dim (int): Dimension of output

    Returns:
        layers (list): List of (layer_type, params)
        params (list): List of parameter dicts
    """
    dims = [input_dim] + hidden_dims + [output_dim]
    layers, params = [], []
    for i in range(len(dims) - 1):
        p = initialize_linear_layer(dims[i], dims[i+1])
        layers.append(("linear", p))
        params.append(p)
        if i < len(dims) - 2:
            layers.append(("relu", None))
    return layers, params

## 6  Whole-network forward & backward

In [ ]:
def mlp_forward(x, layers):
    """
    Forward pass through the full MLP.

    Parameters:
        x (ndarray): Input of shape (batch_size, input_dim)
        layers (list): List of (layer_type, params) tuples

    Returns:
        output (ndarray): Output of the network, shape (batch_size, output_dim)
        caches (list): Cached intermediate values for backward pass
    """
    caches = []
    for layer_type, params in layers:
        if layer_type == "linear":
            x, cache = linear_forward(x, params)
        else:
            x, cache = relu_forward(x)
        caches.append((layer_type, cache))
    return x, caches

def mlp_backward(dout, caches):
    """
    Backward pass through the full MLP.

    Parameters:
        dout (ndarray): Gradient from final loss layer, shape (batch_size, output_dim)
        caches (list): Cached forward pass values

    Returns:
        None (updates gradients in-place)
    """
    for layer_type, cache in reversed(caches):
        if layer_type == "linear":
            dout = linear_backward(dout, cache)
        else:
            dout = relu_backward(dout, cache)

## 7  Synthetic sanity check
In this final part, you'll train your full MLP model on a small synthetic regression task. The input data X is randomly generated, and the targets Y are computed using a known linear transformation. Your goal is to learn the weights of this transformation using a neural network. You'll start by setting hyperparameters like the learning rate, number of epochs, batch size, and hidden layer sizes. The training loop performs standard mini-batch gradient descent: it shuffles the data, runs forward and backward passes through the network, updates weights using the gradients, and prints out the training loss every few epochs. At the end, you'll check that the model has learned well by verifying that the final mean squared error is sufficiently small.

In [ ]:
# Generate synthetic regression data
N, D, C = 20, 5, 3
X = np.random.randn(N, D)
true_W = np.random.randn(C, D)
Y = X @ true_W.T

# TODO-8: set hyperparameters
# === Your code here ===
lr, epochs = ...
batch_size = ...
hidden_dimensions = [...]  # Example hidden layer size, can be a list like [1,2,3,...]
# === Your code end ===

# Build the MLP
layers, params = build_mlp(D, hidden_dimensions, C)

for ep in range(epochs):
    # Shuffle data
    indices = np.random.permutation(N)
    X_shuffled, Y_shuffled = X[indices], Y[indices]

    for i in range(0, N, batch_size):
        x_batch = X_shuffled[i:i+batch_size]
        y_batch = Y_shuffled[i:i+batch_size]

        # TODO-9: forward pass, loss computation (use previously defined functions)
        # === Your code here ===
        y_hat, caches = ...
        loss, diff = ...
        # === Your code end ===

        # Backward pass
        dL = mse_backward(diff)
        mlp_backward(dL, caches)

        sgd_step(params, lr)

        # Zero out gradients for the next iteration
        zero_grads(params)

    if ep % 20 == 0:
        print(f"epoch {ep:03d}  loss {loss:.4e}")

# Evaluate final loss
final = ((mlp_forward(X, layers)[0] - Y) ** 2).mean()
print("final MSE:", final)

# Assert the loss is sufficiently small
assert final < 1e-3, f"MSE too high: {final:.4e} (expected < 1e-3)"
print("✅ Training reached MSE < 1e-3")

With every TODO filled, **final MSE < 1e-3**.

## 8 From PyTorch to NumPy – Understanding the Moving Parts of an MLP

PyTorch is a deep learning framework that automates much of what we've done manually in NumPy.
Now that we’ve built a full MLP (multi-layer perceptron) from scratch using only NumPy, let’s pause and appreciate what we’ve accomplished, while contrasting it with how things are done in PyTorch.

NumPy gives you full control: you compute everything manually. This helps you learn the math and logic behind backpropagation.

PyTorch automates almost everything, making it faster and more readable, especially for larger models or real-world tasks. Once you understand the fundamentals, using PyTorch lets you focus on higher-level ideas, such as:

* Experimenting with architectures
* Trying different optimizers or loss functions without writing complex derivatives every time
* Scaling up to real-world problems

In short: PyTorch automatically tracks computations, builds a computational graph, and performs backpropagation under the hood. You only need to define your model, loss, and optimizer, and PyTorch handles the rest.


| Component            | PyTorch                                 | NumPy (our code)                            |
| -------------------- | --------------------------------------- | ------------------------------------------- |
| Layer definitions    | `torch.nn.Linear(...)`                  | `initialize_linear_layer(...)`              |
| Activation functions | `torch.nn.ReLU()`                       | `relu_forward(...)`                         |
| Forward pass         | `model(x)` or `Sequential.forward(...)` | `mlp_forward(...)`                          |
| Backward gradients   | `loss.backward()`                       | `mlp_backward(...)`, `linear_backward(...)` |
| Parameter updates    | `optimizer.step()`                      | `sgd_step(...)`                             |
| Zeroing gradients    | `optimizer.zero_grad()`                 | `zero_grads(...)`                           |
| Loss computation     | `torch.nn.MSELoss()`                    | `mse_forward(...)`, `mse_backward(...)`     |


Let’s run a this example, a PyTorch version of our code (it's so much shorter!) and try to understand how to use it correctly. PyTorch will make our life easier in tasks 2 and 3 of the project.

In [ ]:
import torch

# Convert our existing data to PyTorch
X_tensor = torch.from_numpy(X).float()
Y_tensor = torch.from_numpy(Y).float()

# Define the MLP 
class TorchMLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(5, 64),     # match input_dim = 5
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 3)      # match output_dim = 3
        )

    def forward(self, x):
        return self.net(x)

# Create model
model = TorchMLP()
print(model)

# Define loss and optimizer
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(1, 400):
    y_pred = model(X_tensor)
    loss = loss_fn(y_pred, Y_tensor)

    optimizer.zero_grad() # Zero gradients
    loss.backward() # Compute gradients
    optimizer.step() # Update parameters (Gradient Descent)

    # Print loss every 20 epochs or at the first epoch
    if epoch % 20 == 0 or epoch == 1:
        print(f"epoch {epoch:03d} | loss {loss.item():.4e}")

# Final check
final_loss = loss_fn(model(X_tensor), Y_tensor).item()
print(f"\nFinal MSE: {final_loss:.4e}")

assert final_loss < 1e-3, f"MSE too high: {final_loss:.4e}"
print("✅ PyTorch model reached MSE < 1e-3")